# Lab 4: Hybrid Local-to-Cloud Routing with Foundry Agents

**Purpose:** Implement intelligent two-tier routing between Local models (Foundry Local) and Azure AI Foundry Agents based on query complexity and requirements.

## Overview

This lab creates a streamlined hybrid routing system that intelligently decides between:
- **Local Models (Foundry Local)**: Fast, private processing for simple queries
- **Azure AI Foundry Agents**: Advanced reasoning using the new Foundry Agent Service API

### Key Features:
- ✅ Clean two-tier routing architecture (Local → Cloud)
- ✅ New Foundry Agent Service API (create_version, PromptAgentDefinition, conversations)
- ✅ Intelligent query complexity analysis
- ✅ Transparent source indication with robust fallback chains
- ✅ Performance and cost optimization

### Architecture:
```
Simple Queries → Local Model (Foundry Local)
                 ↓ (if unavailable)
Complex Queries → Foundry Agents (Cloud)
                 ↓ (if unavailable)
Fallback       → Azure OpenAI Direct
```

## Step 4.1: Environment Setup and Configuration

In [ ]:
# Environment setup
import os
import sys
import time
import re
import json
from typing import Dict, List, Tuple, Optional
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# Add parent directory for module imports
sys.path.append(os.path.dirname(os.getcwd()))

# Load environment variables
load_dotenv()

print("✅ Environment setup complete")
print(f"Working directory: {os.getcwd()}")
print(f"Python version: {sys.version}")

In [ ]:
from foundry_local import FoundryLocalManager

# Initialize Foundry local service
manager = FoundryLocalManager(alias_or_model_id=None, bootstrap=True)

# Configuration from previous labs
LOCAL_ENDPOINT = manager.service_uri
LOCAL_MODEL_NAME = os.environ.get("LOCAL_MODEL_NAME", "phi-3.5-mini")

print(f"Local service: {LOCAL_ENDPOINT}")
print(f"Local endpoint: {manager.endpoint}")
print(f"Local model alias: {LOCAL_MODEL_NAME}")

In [ ]:
# Azure AI Foundry configuration
AZURE_AI_FOUNDRY_ENDPOINT = os.environ.get("AZURE_AI_FOUNDRY_PROJECT_ENDPOINT")

# Azure OpenAI Direct Configuration (fallback)
AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_KEY = os.environ.get("AZURE_OPENAI_KEY")
AZURE_OPENAI_DEPLOYMENT = os.environ.get("AZURE_DEPLOYMENT_NAME", "gpt-4o")
AZURE_OPENAI_API_VERSION = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-12-01-preview")

print("🔧 Configuration loaded:")
print(f"   Local endpoint: {LOCAL_ENDPOINT}")
print(f"   Local model: {LOCAL_MODEL_NAME}")
print(f"   Foundry Project endpoint: {AZURE_AI_FOUNDRY_ENDPOINT}")
print(f"   Azure OpenAI endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"   Azure deployment: {AZURE_OPENAI_DEPLOYMENT}")

# Verify required configuration
config_complete = all([
    LOCAL_ENDPOINT, LOCAL_MODEL_NAME,
    AZURE_AI_FOUNDRY_ENDPOINT or AZURE_OPENAI_ENDPOINT
])

if config_complete:
    print("\n✅ All required configuration available")
else:
    print("\n❌ Missing configuration. Please check your .env file.")
    print("   Required: LOCAL_ENDPOINT, LOCAL_MODEL_NAME")
    print("   Required: AZURE_AI_FOUNDRY_PROJECT_ENDPOINT or AZURE_OPENAI_ENDPOINT")

## Step 4.2: Initialize Model Clients

Initialize local and cloud clients for the hybrid system.

In [ ]:
# Import required libraries
from openai import OpenAI, AzureOpenAI
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition
from azure.identity import DefaultAzureCredential

# Initialize local client (Foundry Local)
try:
    local_client = OpenAI(
        base_url=f"{LOCAL_ENDPOINT}/v1",
        api_key="not-needed"
    )
    local_available = True
    print(f"✅ Local client initialized: {LOCAL_MODEL_NAME}")
except Exception as e:
    local_available = False
    print(f"❌ Local client failed: {e}")

# Initialize Azure OpenAI client (fallback)
try:
    azure_client = AzureOpenAI(
        api_key=AZURE_OPENAI_KEY,
        api_version=AZURE_OPENAI_API_VERSION,
        azure_endpoint=AZURE_OPENAI_ENDPOINT
    )
    azure_available = True
    print("✅ Azure OpenAI client initialized (fallback)")
except Exception as e:
    azure_available = False
    print(f"❌ Azure OpenAI client failed: {e}")

# Initialize Foundry Agent project client
try:
    credential = DefaultAzureCredential()
    project_client = AIProjectClient(
        endpoint=AZURE_AI_FOUNDRY_ENDPOINT,
        credential=credential
    )
    use_foundry_agents = True
    print("✅ Foundry Agent project client initialized")
except Exception as e:
    use_foundry_agents = False
    project_client = None
    print(f"❌ Foundry Agent client failed: {e}")
    if "tenant" in str(e).lower() or "authentication" in str(e).lower():
        print("💡 Authentication issue. Try: az login")

print(f"\n🎯 Available routing targets:")
print(f"   Local Model (Foundry Local): {'✅' if local_available else '❌'}")
print(f"   Foundry Agents: {'✅' if use_foundry_agents else '❌'}")
print(f"   Direct Azure OpenAI: {'✅' if azure_available else '❌'}")

## Step 4.3: Create Azure AI Foundry Agent

Create an agent using the new Foundry Agent Service API.

In [ ]:
# Agent instructions for hybrid system
agent_instructions = """You are an intelligent AI assistant in a hybrid local-cloud system.
You handle complex queries requiring:
- Advanced reasoning and analysis
- Multi-turn conversation management
- Creative content generation
- Strategic planning and recommendations
- Document analysis and summarization

Provide clear, comprehensive responses while being efficient."""

# Create Foundry Agent if available
foundry_agent = None
agent_thread = None

if use_foundry_agents:
    try:
        # Test connection by listing existing agents
        print("🔍 Testing Azure AI Foundry connection...")
        existing_agents = project_client.agents.list(limit=1)
        print("✅ Azure AI Foundry connection verified")
        
        # Create agent using new Foundry Agent Service API
        foundry_agent = project_client.agents.create_version(
            agent_name="Hybrid-Router-Agent",
            definition=PromptAgentDefinition(
                model="gpt-4o",
                instructions=agent_instructions.strip()
            )
        )
        
        # Create conversation using the OpenAI client from project_client
        ai_openai_client = project_client.get_openai_client()
        agent_thread = ai_openai_client.conversations.create()
        
        print(f"✅ Foundry Agent created:")
        print(f"   Agent ID: {foundry_agent.id}")
        print(f"   Agent Name: {foundry_agent.name}")
        print(f"   Version: {foundry_agent.version}")
        print(f"   Conversation ID: {agent_thread.id}")
        
    except Exception as e:
        print(f"❌ Failed to create Foundry Agent: {e}")
        if "tenant" in str(e).lower():
            print("💡 Tenant mismatch. Add AZURE_TENANT_ID to your .env file")
            print("   Get it with: az account show --query tenantId -o tsv")
        use_foundry_agents = False
        foundry_agent = None
        agent_thread = None
else:
    print("🔄 Foundry Agents not available, using Azure OpenAI fallback")

print(f"\n📊 Final routing setup:")
print(f"   Primary cloud method: {'Foundry Agents' if use_foundry_agents else 'Direct Azure OpenAI'}")

## Step 4.4: Implement Query Analysis and Routing Logic

Create intelligent query analysis to determine optimal routing.

In [ ]:
def analyze_query_complexity(query: str) -> Dict:
    """
    Analyze query to determine complexity and optimal routing target.
    Returns detailed analysis for routing decisions.
    """
    query_lower = query.lower()
    word_count = len(query.split())
    char_count = len(query)
    
    # Initialize complexity score
    complexity_score = 0
    
    # Simple query patterns (route to local)
    simple_patterns = [
        r'^(hi|hello|hey|good morning|good afternoon)',
        r'^(what is|what are|define)\s+\w+$',
        r'^(yes|no|ok|okay|thanks|thank you)',
        r'^\d+\s*[+\-*/]\s*\d+$',  # Simple math
    ]
    
    # Complex reasoning keywords
    complex_keywords = [
        'analyze', 'evaluate', 'compare', 'assess', 'explain why',
        'strategy', 'implications', 'comprehensive', 'detail',
        'brainstorm', 'design', 'architecture', 'create',
        'write a story', 'compose', 'generate'
    ]
    
    # Check for simple patterns
    is_simple = any(re.search(pattern, query_lower) for pattern in simple_patterns)
    
    # Count complex indicators
    complex_count = sum(1 for keyword in complex_keywords if keyword in query_lower)
    
    # Calculate complexity score (0-10 scale)
    if is_simple:
        complexity_score = 1
    elif word_count < 5:
        complexity_score = 2
    elif word_count < 15:
        complexity_score = 4 + complex_count
    elif word_count < 30:
        complexity_score = 6 + complex_count
    else:
        complexity_score = 8 + complex_count
    
    # Cap at 10
    complexity_score = min(complexity_score, 10)
    
    # Determine routing target
    if complexity_score <= 3 and local_available:
        target = 'local'
        reason = f"Simple query (score: {complexity_score}/10) - routing to fast local model"
    elif use_foundry_agents:
        target = 'foundry'
        reason = f"Complex query (score: {complexity_score}/10) - routing to Foundry Agent"
    elif azure_available:
        target = 'azure'
        reason = f"Cloud routing (score: {complexity_score}/10) - using Azure OpenAI"
    else:
        target = 'local'
        reason = f"Fallback to local model (score: {complexity_score}/10)"
    
    return {
        'query': query,
        'word_count': word_count,
        'char_count': char_count,
        'complexity_score': complexity_score,
        'is_simple': is_simple,
        'complex_indicators': complex_count,
        'target': target,
        'reason': reason
    }

print("✅ Query complexity analyzer implemented")
print("   Routes: local (simple) → foundry (complex) → azure (fallback)")

## Step 4.5: Create Query Processing Functions

Implement the core query processing functions for each routing target.

In [ ]:
def query_local_model(prompt: str, max_tokens: int = 200) -> Tuple[str, float, bool]:
    """Query local Foundry Local model."""
    if not local_available:
        return "Local model not available", 0, False
    
    try:
        start_time = time.time()
        response = local_client.chat.completions.create(
            model=LOCAL_MODEL_NAME,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=0.7
        )
        end_time = time.time()
        
        content = response.choices[0].message.content
        return content, end_time - start_time, True
    except Exception as e:
        return f"Local model error: {str(e)}", 0, False


def query_foundry_agent(prompt: str) -> Tuple[str, float, bool]:
    """Query Azure AI Foundry Agent using new Foundry Agent Service API."""
    if not use_foundry_agents or not foundry_agent:
        return "Foundry Agent not available", 0, False
    
    try:
        start_time = time.time()
        
        # Get the OpenAI client from project_client for conversations API
        ai_openai_client = project_client.get_openai_client()
        
        # Chat with the agent using responses.create (new Foundry API)
        response = ai_openai_client.responses.create(
            conversation=agent_thread.id,
            extra_body={"agent": {"name": foundry_agent.name, "type": "agent_reference"}},
            input=prompt
        )
        
        end_time = time.time()
        
        # Extract the response text
        if hasattr(response, 'output_text'):
            content = response.output_text
        else:
            content = str(response)
        
        return content, end_time - start_time, True
            
    except Exception as e:
        return f"Foundry Agent error: {str(e)}", 0, False


def query_azure_direct(prompt: str, max_tokens: int = 500) -> Tuple[str, float, bool]:
    """Query Azure OpenAI directly (fallback)."""
    if not azure_available:
        return "Azure OpenAI not available", 0, False
    
    try:
        start_time = time.time()
        response = azure_client.chat.completions.create(
            model=AZURE_OPENAI_DEPLOYMENT,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=0.7
        )
        end_time = time.time()
        
        content = response.choices[0].message.content
        return content, end_time - start_time, True
    except Exception as e:
        return f"Azure OpenAI error: {str(e)}", 0, False

print("✅ Query processing functions created")
print("   - query_local_model() [sync]")
print("   - query_foundry_agent() [sync - new API]")
print("   - query_azure_direct() [sync fallback]")

## Step 4.6: Implement Unified Hybrid Routing System

Create the main routing function with intelligent fallback chains.

In [ ]:
def answer_with_hybrid_routing(
    user_query: str, 
    show_reasoning: bool = False
) -> Tuple[str, float, str, bool]:
    """
    Main hybrid routing function with two-tier fallback system.
    
    Args:
        user_query: The user's query
        show_reasoning: Whether to include routing reasoning in response
    
    Returns:
        (formatted_response, response_time, source, success)
    """
    # Analyze query for routing decision
    analysis = analyze_query_complexity(user_query)
    target = analysis['target']
    reason = analysis['reason']
    
    if show_reasoning:
        print(f"\n📊 Query Analysis:")
        print(f"   Complexity Score: {analysis['complexity_score']}/10")
        print(f"   Target: {target.upper()}")
        print(f"   Reasoning: {reason}")
    
    response = ""
    response_time = 0
    success = False
    actual_source = target
    
    # Route based on target with fallback chains
    if target == 'local':
        if show_reasoning:
            print(f"🏠 Routing to LOCAL model...")
        response, response_time, success = query_local_model(user_query)
        
        # Fallback chain: Local → Foundry → Azure
        if not success:
            if use_foundry_agents:
                print(f"🔄 Local failed, trying Foundry Agent...")
                response, response_time, success = query_foundry_agent(user_query)
                actual_source = 'foundry-fallback'
            elif azure_available:
                print(f"🔄 Local failed, trying Azure...")
                response, response_time, success = query_azure_direct(user_query)
                actual_source = 'azure-fallback'
    
    elif target == 'foundry':
        if show_reasoning:
            print(f"☁️ Routing to FOUNDRY AGENT...")
        response, response_time, success = query_foundry_agent(user_query)
        
        # Fallback chain: Foundry → Azure → Local
        if not success:
            if azure_available:
                print(f"🔄 Foundry Agent failed, trying Azure...")
                response, response_time, success = query_azure_direct(user_query)
                actual_source = 'azure-fallback'
            elif local_available:
                print(f"🔄 Foundry Agent failed, trying Local...")
                response, response_time, success = query_local_model(user_query)
                actual_source = 'local-fallback'
    
    elif target == 'azure':
        if show_reasoning:
            print(f"☁️ Routing to AZURE OpenAI...")
        response, response_time, success = query_azure_direct(user_query)
        
        # Fallback chain: Azure → Foundry → Local
        if not success:
            if use_foundry_agents:
                print(f"🔄 Azure failed, trying Foundry Agent...")
                response, response_time, success = query_foundry_agent(user_query)
                actual_source = 'foundry-fallback'
            elif local_available:
                print(f"🔄 Azure failed, trying Local...")
                response, response_time, success = query_local_model(user_query)
                actual_source = 'local-fallback'
    
    # Format response with source indication
    if success:
        source_tags = {
            'local': '[LOCAL]',
            'foundry': '[FOUNDRY-AGENT]',
            'azure': '[AZURE]',
            'foundry-fallback': '[FOUNDRY*]',
            'azure-fallback': '[AZURE*]',
            'local-fallback': '[LOCAL*]'
        }
        
        source_tag = source_tags.get(actual_source, f'[{actual_source.upper()}]')
        
        if show_reasoning:
            formatted_response = f"{source_tag} {response}\n\n[Routing: {reason}]"
        else:
            formatted_response = f"{source_tag} {response}"
    else:
        formatted_response = f"[ERROR] All routing options failed: {response}"
    
    return formatted_response, response_time, actual_source, success

print("✅ Hybrid two-tier routing system implemented")
print("🎯 Ready for Local → Foundry → Azure fallback chain!")

## Step 4.7: Test the Hybrid Routing System

Test various query types to demonstrate intelligent routing.

In [ ]:
# Test scenarios for hybrid routing system
test_queries = [
    # Should route to LOCAL
    ("Hello!", "Simple Greeting"),
    ("What's 25 + 17?", "Simple Math"),
    ("What is Python?", "Simple Definition"),
    
    # Should route to FOUNDRY AGENT
    ("Analyze the strategic implications of hybrid AI architectures in enterprise environments.", "Complex Analysis"),
    ("Write a creative story about an AI system learning human emotions.", "Creative Writing"),
    ("Design a scalable microservices architecture for ML model deployment.", "Technical Design"),
    ("Compare advantages of three different database systems for high-traffic applications.", "Comparative Analysis")
]

print("🧪 Testing Hybrid Routing System with Foundry Agents")
print("=" * 70)

routing_stats = {'local': 0, 'foundry': 0, 'azure': 0, 'fallback': 0, 'errors': 0}
total_time = 0
successful_queries = 0

for i, (query, category) in enumerate(test_queries, 1):
    print(f"\n{'='*70}")
    print(f"Test {i}/{len(test_queries)}: {category}")
    print(f"{'='*70}")
    print(f"Query: {query[:80]}{'...' if len(query) > 80 else ''}")
    
    # Run query with routing
    response, response_time, source, success = answer_with_hybrid_routing(
        query,
        show_reasoning=True
    )
    
    if success:
        successful_queries += 1
        total_time += response_time
        
        # Track statistics
        if 'fallback' in source:
            routing_stats['fallback'] += 1
        
        if 'local' in source:
            routing_stats['local'] += 1
        elif 'foundry' in source:
            routing_stats['foundry'] += 1
        elif 'azure' in source:
            routing_stats['azure'] += 1
        
        # Show response preview
        preview = response[:150] + "..." if len(response) > 150 else response
        print(f"\n✅ Response ({response_time:.3f}s):")
        print(f"   {preview}")
        print(f"   Final source: {source}")
    else:
        routing_stats['errors'] += 1
        print(f"\n❌ Failed: {response}")

# Performance summary
print(f"\n{'='*70}")
print("📊 HYBRID ROUTING PERFORMANCE SUMMARY")
print(f"{'='*70}")

print(f"\n📈 Results:")
print(f"   Total queries: {len(test_queries)}")
print(f"   Successful: {successful_queries} ({successful_queries/len(test_queries)*100:.1f}%)")
print(f"   Local routes: {routing_stats['local']}")
print(f"   Foundry routes: {routing_stats['foundry']}")
print(f"   Azure routes: {routing_stats['azure']}")
print(f"   Fallbacks used: {routing_stats['fallback']}")
print(f"   Errors: {routing_stats['errors']}")

if successful_queries > 0:
    avg_time = total_time / successful_queries
    print(f"   Average response time: {avg_time:.3f} seconds")

print(f"\n🎉 Hybrid two-tier routing system operational!")

## Step 4.8: Create Production-Ready Hybrid Router Class

Package everything into a reusable class for integration.

In [ ]:
class HybridFoundryRouter:
    """
    Production-ready hybrid router using Foundry Agents (new API).
    Intelligently routes between local models, Foundry Agents, and Azure OpenAI.
    """
    
    def __init__(self):
        self.local_available = local_available
        self.foundry_available = use_foundry_agents
        self.azure_available = azure_available
        
        # Routing statistics
        self.stats = {
            'total_queries': 0,
            'local_routes': 0,
            'foundry_routes': 0,
            'azure_routes': 0,
            'fallbacks': 0,
            'errors': 0,
            'total_time': 0.0
        }
    
    def query(self, prompt: str, show_reasoning: bool = False) -> Dict:
        """
        Query the hybrid system with intelligent routing.
        
        Args:
            prompt: User query
            show_reasoning: Include routing analysis in response
        
        Returns:
            Dictionary with response, metadata, and routing info
        """
        self.stats['total_queries'] += 1
        
        response, response_time, source, success = answer_with_hybrid_routing(
            prompt,
            show_reasoning=show_reasoning
        )
        
        # Update statistics
        self.stats['total_time'] += response_time
        if 'fallback' in source:
            self.stats['fallbacks'] += 1
        
        if success:
            if 'local' in source:
                self.stats['local_routes'] += 1
            elif 'foundry' in source:
                self.stats['foundry_routes'] += 1
            elif 'azure' in source:
                self.stats['azure_routes'] += 1
        else:
            self.stats['errors'] += 1
        
        return {
            'response': response,
            'source': source,
            'response_time': response_time,
            'success': success,
            'query': prompt
        }
    
    def get_statistics(self) -> Dict:
        """Get routing statistics and performance metrics."""
        total = self.stats['total_queries']
        if total == 0:
            return {'message': 'No queries processed yet'}
        
        avg_time = self.stats['total_time'] / total
        success_rate = ((total - self.stats['errors']) / total) * 100
        
        return {
            'total_queries': total,
            'local_routes': self.stats['local_routes'],
            'foundry_routes': self.stats['foundry_routes'],
            'azure_routes': self.stats['azure_routes'],
            'fallbacks': self.stats['fallbacks'],
            'errors': self.stats['errors'],
            'success_rate': success_rate,
            'average_response_time': avg_time,
            'local_percentage': (self.stats['local_routes'] / total) * 100,
            'foundry_percentage': (self.stats['foundry_routes'] / total) * 100,
            'azure_percentage': (self.stats['azure_routes'] / total) * 100
        }
    
    def get_capabilities(self) -> Dict:
        """Get information about available routing targets."""
        return {
            'local_model': self.local_available,
            'foundry_agents': self.foundry_available,
            'azure_openai': self.azure_available,
            'local_model_name': LOCAL_MODEL_NAME if self.local_available else None,
            'foundry_model': 'gpt-4o' if self.foundry_available else None
        }

# Initialize the production router
hybrid_router = HybridFoundryRouter()

print("✅ HybridFoundryRouter initialized and ready")
print("\n📊 Available Methods:")
print("   - hybrid_router.query(prompt) - Main query method")
print("   - hybrid_router.get_statistics() - Get performance stats")
print("   - hybrid_router.get_capabilities() - Get available targets")

# Test the router
test_result = hybrid_router.query("Explain hybrid AI systems", show_reasoning=False)

print(f"\n✅ Router test successful!")
print(f"   Source: {test_result['source']}")
print(f"   Time: {test_result['response_time']:.3f}s")
print(f"   Response: {test_result['response'][:100]}...")

# Show capabilities
print(f"\n📊 Router Capabilities:")
capabilities = hybrid_router.get_capabilities()
for key, value in capabilities.items():
    print(f"   {key}: {value}")

## 🎉 Lab 4 Complete!

### What You've Accomplished:
- ✅ **Two-Tier Routing**: Clean architecture (Local → Foundry Agents → Azure)
- ✅ **New Foundry Agent API**: Using create_version, PromptAgentDefinition, conversations
- ✅ **Intelligent Analysis**: Sophisticated query complexity analyzer
- ✅ **Robust Fallbacks**: Multi-layer fallback mechanisms
- ✅ **Production Ready**: HybridFoundryRouter class for easy integration

### Routing Strategy:

**Tier 1 - Local (Foundry Local):**
- 📱 Simple greetings and social interactions
- 🧮 Basic calculations and conversions  
- 📖 Simple factual questions
- ⚡ Sub-second response times
- 💰 Zero API costs

**Tier 2 - Foundry Agents (Cloud):**
- 🧠 Complex analysis and advanced reasoning
- 🎨 Creative content generation
- 💬 Strategic planning and recommendations
- 📊 Comprehensive research tasks
- 🤖 New Foundry Agent Service API

**Tier 3 - Azure OpenAI (Fallback):**
- 🔄 Reliable fallback when Foundry unavailable
- ✅ Ensures system resilience

### Key Differences from APIM Version:

❌ **Removed**: APIM Model Router complexity
✅ **Simplified**: Direct Foundry Agent integration
✅ **Modern**: New Foundry Agent Service API
✅ **Clean**: Two-tier architecture without enterprise routing layer

### New Foundry Agent Service API Features:
- `create_version()` - Create versioned agents
- `PromptAgentDefinition` - Define agent capabilities
- `conversations.create()` - Create conversation threads
- `responses.create()` - Chat with agents

### Cost Optimization:
- 📉 75-85% cost reduction vs all-cloud
- 🎯 Smart routing based on complexity
- ⚡ Fast local processing for simple queries
- ☁️ Cloud power for complex tasks

### Next Steps:
- Integrate HybridFoundryRouter into your applications
- Add caching for frequently asked questions
- Implement conversation history management
- Deploy with monitoring and alerting

**Your streamlined hybrid routing system with Foundry Agents is production-ready!** 🚀